### Merge pheno with peidg files 

- Build a scrubbing file to be used on subtype analysis for wm task
- Add mean run1 and run1 for FD an FD_scrubbed
- Visualize correlation between run 1 and 2 for FD and FD scrubbed
- merge clustered phenotype with scrubbing

In [34]:
# Import pandas
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display

In [35]:
%matplotlib inline

In [36]:
# Set path
#path_root = "~/Drive/HCP/"
path_root = "~/data/data_disk/Drive/HCP2/"
path_pheno = os.path.join(path_root,"subtypes_scores/26-10-2016/pheno")
GenMetSub_file = os.path.join(path_pheno,"GenMetSub.csv")
unrestric_file = os.path.join(path_pheno,"unrestricted_yuzhang_4_16_2019_10_51_46.csv")
restric_file = os.path.join(path_pheno,"RESTRICTED_yuzhang_4_16_2019_10_53_20.csv")

In [37]:
# Merge restricted and un restricted files 
pheno_pedig_raw = pd.merge(pd.read_csv(restric_file),pd.read_csv(unrestric_file),how='left',on='Subject')
pheno_pedig_raw.head()

,Subject,Age_in_Yrs,HasGT,ZygositySR,ZygosityGT,Family_ID,Mother_ID,Father_ID,TestRetestInterval,Race,...,Noise_Comp,Odor_Unadj,Odor_AgeAdj,PainIntens_RawScore,PainInterf_Tscore,Taste_Unadj,Taste_AgeAdj,Mars_Log_Score,Mars_Errs,Mars_Final
0,100004,24,True,NotTwin,,52259_82122,52259,82122,NaN,White,...,5.2,101.12,86.45,2.0,45.9,107.17,105.31,1.80,0.0,1.80
1,100206,27,True,NotTwin,,56037_85858,56037,85858,NaN,White,...,6.0,108.79,97.19,1.0,49.7,72.63,72.03,1.84,0.0,1.84
2,100307,27,True,NotMZ,MZ,51488_81352,51488,81352,NaN,White,...,3.6,101.12,86.45,0.0,38.6,71.69,71.76,1.76,0.0,1.76
3,100408,33,True,MZ,MZ,51730_81594,51730,81594,NaN,White,...,2.0,108.79,98.04,2.0,52.6,114.01,113.59,1.76,2.0,1.68
4,100610,27,True,NotMZ,DZ,52813_82634,52813,82634,NaN,White,...,2.0,122.25,110.45,0.0,38.6,84.84,85.31,1.92,1.0,1.88


In [38]:
# Merge restricted and un restricted files 
pheno_pedig_raw = pd.merge(pd.read_csv(restric_file),pd.read_csv(unrestric_file),how='left',on='Subject')

# create new zigosity column that is a merge between self repoerted and un update from the genotyping info 
list_=[]
for ind,val in pheno_pedig_raw.iterrows():
    if val.ZygosityGT == " ":
        list_.append(val.ZygositySR)
    elif val.ZygosityGT == val.ZygositySR:
        list_.append(val.ZygosityGT)
    elif (val.ZygosityGT == 'MZ' and (val.ZygositySR == 'NotMZ' or val.ZygositySR == ' ')):
        list_.append(val.ZygosityGT)
    elif val.ZygosityGT == 'DZ' and val.ZygositySR == 'NotMZ':
         list_.append(val.ZygositySR)

pheno_pedig_raw.insert(loc=5, column='ZygositySG', value=list_)
pheno_pedig_raw.head()

,Subject,Age_in_Yrs,HasGT,ZygositySR,ZygosityGT,ZygositySG,Family_ID,Mother_ID,Father_ID,TestRetestInterval,...,Noise_Comp,Odor_Unadj,Odor_AgeAdj,PainIntens_RawScore,PainInterf_Tscore,Taste_Unadj,Taste_AgeAdj,Mars_Log_Score,Mars_Errs,Mars_Final
0,100004,24,True,NotTwin,,NotTwin,52259_82122,52259,82122,NaN,...,5.2,101.12,86.45,2.0,45.9,107.17,105.31,1.80,0.0,1.80
1,100206,27,True,NotTwin,,NotTwin,56037_85858,56037,85858,NaN,...,6.0,108.79,97.19,1.0,49.7,72.63,72.03,1.84,0.0,1.84
2,100307,27,True,NotMZ,MZ,MZ,51488_81352,51488,81352,NaN,...,3.6,101.12,86.45,0.0,38.6,71.69,71.76,1.76,0.0,1.76
3,100408,33,True,MZ,MZ,MZ,51730_81594,51730,81594,NaN,...,2.0,108.79,98.04,2.0,52.6,114.01,113.59,1.76,2.0,1.68
4,100610,27,True,NotMZ,DZ,NotMZ,52813_82634,52813,82634,NaN,...,2.0,122.25,110.45,0.0,38.6,84.84,85.31,1.92,1.0,1.88


In [39]:
merged_pheno = pd.merge(pheno_pedig_raw[['Subject','Age_in_Yrs','Gender','ZygositySG','Mother_ID','BMI']],
                        pd.read_csv(GenMetSub_file), on='Subject', how='right')
merged_pheno.head()

,Subject,Age_in_Yrs,Gender,ZygositySG,Mother_ID,BMI,GeMetSub_1,GeMetSub_2,GeMetSub_3,GeMetSub_4,GeMetSub_5,GeMetSub_6,GeMetSub_7,GeMetSub_8
0,100206,27,M,NotTwin,56037,26.64,0.0020,0.009000,-0.0220,-0.0246,0.265333,-0.243000,0.072333,-0.018263
1,100307,27,F,MZ,51488,22.96,0.0920,0.361323,-0.2100,-0.1980,0.064667,-0.182000,-0.304667,-0.395868
2,100408,33,M,MZ,51730,27.75,-0.0004,-0.306903,-0.0290,-0.0046,0.203667,0.092667,0.093333,0.355632
3,100610,27,M,NotMZ,52813,36.91,0.0816,0.028000,0.4815,0.2388,-0.048667,-0.027667,-0.194333,-0.066500
4,101006,35,F,MZ,51283,30.27,-0.0532,-0.013742,-0.3600,-0.1454,-0.102000,0.096333,0.074333,0.040711


In [40]:
# remove rows with missing values
missing = np.array([merged_pheno[col].isnull().values for col in merged_pheno.columns.get_values()]).any(0)
merged_pheno_clean = merged_pheno[~missing]
merged_pheno_clean.count()

Subject       726
Age_in_Yrs    726
Gender        726
ZygositySG    726
Mother_ID     726
BMI           726
GeMetSub_1    726
GeMetSub_2    726
GeMetSub_3    726
GeMetSub_4    726
GeMetSub_5    726
GeMetSub_6    726
GeMetSub_7    726
GeMetSub_8    726
dtype: int64

In [41]:
# set gender to numerical
merged_pheno_clean.loc[:,'Gender']= merged_pheno_clean.Gender.replace({'M':1,'F':2})
merged_pheno_clean.loc[:,'Subject']= 'HCP' + merged_pheno_clean['Subject'].astype(str)

In [42]:
merged_pheno_clean.head()

,Subject,Age_in_Yrs,Gender,ZygositySG,Mother_ID,BMI,GeMetSub_1,GeMetSub_2,GeMetSub_3,GeMetSub_4,GeMetSub_5,GeMetSub_6,GeMetSub_7,GeMetSub_8
0,HCP100206,27,1,NotTwin,56037,26.64,0.0020,0.009000,-0.0220,-0.0246,0.265333,-0.243000,0.072333,-0.018263
1,HCP100307,27,2,MZ,51488,22.96,0.0920,0.361323,-0.2100,-0.1980,0.064667,-0.182000,-0.304667,-0.395868
2,HCP100408,33,1,MZ,51730,27.75,-0.0004,-0.306903,-0.0290,-0.0046,0.203667,0.092667,0.093333,0.355632
3,HCP100610,27,1,NotMZ,52813,36.91,0.0816,0.028000,0.4815,0.2388,-0.048667,-0.027667,-0.194333,-0.066500
4,HCP101006,35,2,MZ,51283,30.27,-0.0532,-0.013742,-0.3600,-0.1454,-0.102000,0.096333,0.074333,0.040711


In [43]:
##save to csv
merged_pheno_clean.rename(columns={'Subject':'ID'}).to_csv(os.path.join(path_pheno,"GenMetSub_pheno.csv"), index=False)

In [44]:
# List of network names
list_trial = pd.read_csv(os.path.join(GenMetSub_file)).columns.drop('Subject')
list_trial = list_trial.tolist()
list_trial

['GeMetSub_1',
 'GeMetSub_2',
 'GeMetSub_3',
 'GeMetSub_4',
 'GeMetSub_5',
 'GeMetSub_6',
 'GeMetSub_7',
 'GeMetSub_8']

## Build trait file

In [45]:
# Remove not neaded traits
list_trait = merged_pheno_clean.columns.tolist()
new_list = [trait for trait in list_trait if trait not in ['Subject','Age_in_Yrs','Gender','ZygositySG','Mother_ID']]
new_list

['BMI',
 'GeMetSub_1',
 'GeMetSub_2',
 'GeMetSub_3',
 'GeMetSub_4',
 'GeMetSub_5',
 'GeMetSub_6',
 'GeMetSub_7',
 'GeMetSub_8']

## Build Pedigree file

In [46]:
# Select a subset of variable from pheno file
file_pheno_pedig = merged_pheno_clean[["Subject","ZygositySG","Mother_ID","Gender"]]
file_pheno_pedig.rename(columns={'Subject':'ID'},inplace=True)
file_pheno_pedig.head()

/home/yassinebha/jupyter_py3.0/lib/python3.5/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,ID,ZygositySG,Mother_ID,Gender
0,HCP100206,NotTwin,56037,1
1,HCP100307,MZ,51488,2
2,HCP100408,MZ,51730,1
3,HCP100610,NotMZ,52813,1
4,HCP101006,MZ,51283,2


In [47]:
file_pheno_pedig.ZygositySG.unique()

array(['NotTwin', 'MZ', 'NotMZ'], dtype=object)

In [48]:
# Build pedigre : ID , mo, fo, hhID, mztwins,sex
mother_id = ["mo_{}".format(row.Mother_ID) for ind, row in file_pheno_pedig.iterrows()]
father_id = ["fa_{}".format(row.Mother_ID) for ind, row in file_pheno_pedig.iterrows()]
household_id = ["hh_{}".format(row.Mother_ID) for ind, row in file_pheno_pedig.iterrows()]
twin_id = ["pair_{}".format(row.Mother_ID) if row.ZygositySG == "MZ" else "" for ind, row in file_pheno_pedig.iterrows()]

In [49]:
# write into new data frame
df_pedig = pd.DataFrame()
df_pedig["ID"] = file_pheno_pedig.ID
df_pedig["fa"] = father_id
df_pedig["mo"] = mother_id
df_pedig["sex"] = file_pheno_pedig.Gender.astype(int)
df_pedig["mztwin"] = twin_id
df_pedig["hhID"] = household_id


print("head")
display(df_pedig.head())
print("tail")
display(df_pedig.tail())
df_pedig.count()

head


,ID,fa,mo,sex,mztwin,hhID
0,HCP100206,fa_56037,mo_56037,1,,hh_56037
1,HCP100307,fa_51488,mo_51488,2,pair_51488,hh_51488
2,HCP100408,fa_51730,mo_51730,1,pair_51730,hh_51730
3,HCP100610,fa_52813,mo_52813,1,,hh_52813
4,HCP101006,fa_51283,mo_51283,2,pair_51283,hh_51283


tail


,ID,fa,mo,sex,mztwin,hhID
721,HCP992673,fa_56143,mo_56143,2,,hh_56143
722,HCP992774,fa_51345,mo_51345,1,,hh_51345
723,HCP993675,fa_55800,mo_55800,2,,hh_55800
724,HCP994273,fa_52364,mo_52364,1,,hh_52364
725,HCP996782,fa_51541,mo_51541,2,,hh_51541


ID        726
fa        726
mo        726
sex       726
mztwin    726
hhID      726
dtype: int64

In [50]:
# Look for MZ twins missing their co-twin
uniques = list(df_pedig.mztwin.unique())
uniques.pop(0)
twin_counts = [np.sum(df_pedig.mztwin==twin) for twin in uniques]
# for idx, twin in enumerate(uniques):
#     print("{} occurs {} times".format(twin, twin_counts[idx]))

In [51]:
# create a mask that will kepp only MZ twins missing their co-twin
singles = [True if np.sum(df_pedig.mztwin==row.mztwin)==1 else False for ind, row in df_pedig.iterrows()]

In [52]:
# Make a copy of the pedigre to avoid missing up with the original data
copy_df_pedig = df_pedig.copy()

In [53]:
# Select only MZ twins missing their co-twin
df_single_pedig = copy_df_pedig[singles]

# Add "M_" (for missing) to a the single MZ twins
for i, row in df_single_pedig.iterrows():
    df_single_pedig.set_value(i,"ID","M_{}".format(str(row.ID)))
display(df_single_pedig.head())
df_single_pedig.count()

/home/yassinebha/jupyter_py3.0/lib/python3.5/site-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


,ID,fa,mo,sex,mztwin,hhID
34,M_HCP110007,fa_54572,mo_54572,2,pair_54572,hh_54572
36,M_HCP111009,fa_51495,mo_51495,2,pair_51495,hh_51495
37,M_HCP111312,fa_51381,mo_51381,2,pair_51381,hh_51381
50,M_HCP114823,fa_99996,mo_99996,2,pair_99996,hh_99996
54,M_HCP116221,fa_55892,mo_55892,1,pair_55892,hh_55892


ID        54
fa        54
mo        54
sex       54
mztwin    54
hhID      54
dtype: int64

In [54]:
# Append pedgree with fake co-twin for the missing one
df_pedig_final = copy_df_pedig.append(df_single_pedig)
df_pedig_final.head()

,ID,fa,mo,sex,mztwin,hhID
0,HCP100206,fa_56037,mo_56037,1,,hh_56037
1,HCP100307,fa_51488,mo_51488,2,pair_51488,hh_51488
2,HCP100408,fa_51730,mo_51730,1,pair_51730,hh_51730
3,HCP100610,fa_52813,mo_52813,1,,hh_52813
4,HCP101006,fa_51283,mo_51283,2,pair_51283,hh_51283


In [55]:
df_pedig_final.tail()

,ID,fa,mo,sex,mztwin,hhID
651,M_HCP833249,fa_51576,mo_51576,2,pair_51576,hh_51576
652,M_HCP835657,fa_52137,mo_52137,2,pair_52137,hh_52137
654,M_HCP837964,fa_52875,mo_52875,2,pair_52875,hh_52875
691,M_HCP904044,fa_51290,mo_51290,2,pair_51290,hh_51290
713,M_HCP972566,fa_56077,mo_56077,2,pair_56077,hh_56077


In [56]:
df_pedig_final.count()

ID        780
fa        780
mo        780
sex       780
mztwin    780
hhID      780
dtype: int64

In [57]:
# Save the pedigree file
df_pedig_final.to_csv(os.path.join(path_pheno,'GenMetSub_pedigre.csv'), index=False)